Установка необходимых библиотек

In [ ]:
%%capture
!pip install tensorflow_text
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm
!pip install progress

Монтирование диска Google Drive для загрузк и харанения  данных.

**ВНИМАНИЕ** для доступа к данным необходимо выполнить следующие действия:

1. Перейти по [ссылке](https://drive.google.com/drive/folders/1MnoxVXjG8o8kL0SS14E3YzbZmCovsGpR?usp=sharing)
2. Создать ярлык на данную директорию в своём Google Drive. Для этого нажмите на название директории а затем на "Добавить ярлык на Диск".

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Импортирование необходимых библиотек


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

Выбор и загрузка модели для получения эмбеддингов

In [ ]:
# mode, alias = "-large", "large"
mode, alias = "", "original"
module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual{}/3".format(mode)
model = hub.load(module_url)

Подсчёт близости запросов и загоовков документов в пространстве эмбеддингов.

In [ ]:
prefix = Path("/content/gdrive/My Drive/Kononov_NLP")
queries_df = pd.read_csv(prefix / "queries.tsv", sep="\t", index_col=0)
titles_df = pd.read_csv(prefix / "titles.tsv", sep="\t", index_col=0).fillna("")
relation_df = pd.read_csv(prefix / "samples.tsv", sep="\t")

relation_df.drop(columns=["label", "sample_id"], inplace=True)
relation_df = relation_df.assign(embedding_similarity=np.full(relation_df.index.size, 0.0))
for key, group in tqdm(relation_df.groupby("query_id")):
  doc_idx = group["doc_id"].values
  query_embedding = model(queries_df["text"].loc[key])
  title_embeddings = model(titles_df["text"].loc[doc_idx].to_list())
  relation_df.loc[group.index, "embedding_similarity"] = cosine_similarity(query_embedding.numpy().reshape(1, -1),
                                                                           title_embeddings.numpy()).reshape(-1)

path = prefix / "semantic/use_{}_query_doc_embedding_similarity.tsv".format(alias)
relation_df.to_csv(path, sep="\t", index=False)
print("Result saved to file: {}".format(path))